In [55]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
#from autoviz.AutoViz_Class import AutoViz_Class

pas = pd.read_csv('/Users/yiming/Desktop/CoverMyMeds/dim_pa.csv')
pas.head(15)

,dim_pa_id,correct_diagnosis,tried_and_failed,contraindication,pa_approved
0,1,1,1,0,1
1,2,1,0,0,1
2,3,0,0,1,1
3,4,1,1,0,1
4,5,0,1,0,1
5,6,1,1,0,1
6,7,0,1,0,1
7,8,1,0,1,0
8,9,1,0,1,0
9,10,1,1,0,1


In [79]:
claims = pd.read_csv('/Users/yiming/Desktop/CoverMyMeds/dim_claims.csv')
claims.head(15)

,dim_claim_id,bin,drug,reject_code,pharmacy_claim_approved
0,1,417380,A,75.0,0
1,2,999001,A,NaN,1
2,3,417740,A,76.0,0
3,4,999001,A,NaN,1
4,5,417740,A,NaN,1
5,6,417740,A,NaN,1
6,7,999001,A,NaN,1
7,8,417740,A,NaN,1
8,9,999001,A,NaN,1
9,10,417380,A,75.0,0


In [98]:
### Make one-hot predictors with reject_code
reject_code = claims['reject_code'].to_numpy()
nan_array = np.isnan(reject_code)
not_nan_array = ~ nan_array
reject = reject_code[not_nan_array]
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(reject.reshape(-1,1))
reject = enc.transform(reject.reshape(-1,1)).toarray()
print('One-hot labels:',reject[:10,:])

One-hot labels: [[0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [99]:
### Use reject_code,correct_diagnosis,tried_and_failed and contraindication to predict the pa approval rate
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

X = np.hstack((reject,pas[['correct_diagnosis','tried_and_failed','contraindication']].to_numpy()))
y = pas[['pa_approved']].to_numpy()
X = X[:500,:]
y = y[:500,:]
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25,
                                                    random_state = 614, 
                                                    shuffle=True,
                                                    stratify=y)
def get_acc(model,X,y):
    pred = model.predict(X)
    return np.sum(pred == y)/len(y)

j = 0
log_accuracy = []
print('coefficients')
for train_index, test_index in kfold.split(X_train,y_train):
    X_train_train, X_train_test = X[train_index], X[test_index]
    y_train_train, y_train_test = y[train_index], y[test_index]
    clf = LogisticRegression(random_state=0,fit_intercept=True).fit(X_train_train, y_train_train.ravel())
    print(clf.coef_)
    log_accuracy.append(get_acc(clf,X_train_test,y_train_test.ravel()))
    j += 1
print('accuracy',np.mean(log_accuracy))


coefficients
[[-1.71591291  1.57010198  0.14582442  0.31865596  1.21281104 -1.84890768]]
[[-1.71078392  1.52403683  0.1867936   0.46199753  1.33794678 -1.91021562]]
[[-1.84175534  1.66332884  0.1784388   0.11403292  1.29948243 -1.87005422]]
[[-1.71971258  1.45846516  0.26126601  0.3402587   1.20240596 -2.09558529]]
[[-1.81202437  1.6132627   0.19876592  0.1537274   1.18742092 -1.88203419]]
[[-1.67243569  1.52553286  0.14688629  0.54339787  1.31733327 -1.96380998]]
[[-1.72359195  1.45098558  0.27260634  0.3497605   1.16609038 -1.85346047]]
[[-1.81776226  1.42634578  0.39139632  0.43770364  1.33356705 -1.81625247]]
[[-1.96949258  1.45366164  0.51577855  0.22038985  1.19251441 -1.81953304]]
[[-1.72837045  1.5350727   0.19329905  0.23545242  1.1453487  -1.98338358]]
accuracy 0.8051209103840682


In [95]:
## Throw away correct_diagnosis can increase the accuracy
X = np.hstack((reject,pas[['tried_and_failed','contraindication']].to_numpy()))
y = pas[['pa_approved']].to_numpy()
X = X[:500,:]
y = y[:500,:]
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25,
                                                    random_state = 614, 
                                                    shuffle=True,
                                                    stratify=y)
j = 0
log_accuracy = []
for train_index, test_index in kfold.split(X_train,y_train):
    X_train_train, X_train_test = X[train_index], X[test_index]
    y_train_train, y_train_test = y[train_index], y[test_index]
    clf = LogisticRegression(random_state=0,fit_intercept=True).fit(X_train_train, y_train_train.ravel())
    log_accuracy.append(get_acc(clf,X_train_test,y_train_test.ravel()))
    j += 1
print('The accuracy of logistic regression with tried_and_failed and contraindication as predictors is', np.mean(log_accuracy))

The accuracy of logistic regression with tried_and_failed and contraindication as predictors is 0.8105263157894737


In [100]:
### Use K-means to predict the proba of a PA being approved
from sklearn.neighbors import KNeighborsClassifier
j = 0
max_neighbors = 200
kfold = StratifiedKFold(10,shuffle = True,random_state = 440)
cv_accuracy = np.zeros((max_neighbors, 10))

for train_index, test_index in kfold.split(X_train,y_train):
    X_train_train, X_train_test = X[train_index], X[test_index]
    y_train_train, y_train_test = y[train_index], y[test_index]
    for i in range(1,max_neighbors+1):
        knn = KNeighborsClassifier(n_neighbors = i)
        knn.fit(X_train_train,y_train_train.ravel())
        cv_accuracy[i-1,j] = get_acc(knn,X_train_test,y_train_test.ravel())
    j += 1 
cv_accuracy = np.mean(cv_accuracy,axis = 1)
print(np.argmax(cv_accuracy),"- means achieves the best accuracy of",np.max(cv_accuracy))


11 - means achieves the best accuracy of 0.8103129445234709
